In [1]:
import requests
import pandas as pd

api_key = ""

word_dictionary = pd.read_csv("../data/raw/word/word_dictionary.csv", sep=',')

def is_word_registered(word, api_key):
    url = f"https://stdict.korean.go.kr/api/search.do?certkey_no=6879&key={api_key}&type_search=search&req_type=json&q={word}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return len(data['channel'].get('item', [])) > 0
    else:
        print(f"Error: {response.status_code}")
        return False



word_dictionary['definition'] = None

filtered_words = []

error_log = []

i=0
for word in word_dictionary['Word']:
    try:
        if is_word_registered(word, api_key):
            filtered_words.append(word)
            i=i+1
    except Exception as e:
        print(f"Error occurred for word '{word}': {e}")
        error_log.append((word, str(e)))
    if i%50==0:
        print(i)

# Convert filtered words back to DataFrame
filtered_df = word_dictionary[word_dictionary['Word'].isin(filtered_words)]


KeyboardInterrupt: 

In [4]:
error = pd.DataFrame(error_log, columns=['word', 'error'])
print(error)

     word                                      error
0      국활  Expecting value: line 1 column 1 (char 0)
1     바르게  Expecting value: line 1 column 1 (char 0)
2     되어다  Expecting value: line 1 column 1 (char 0)
3      시나  Expecting value: line 1 column 1 (char 0)
4     시르다  Expecting value: line 1 column 1 (char 0)
5      쓴다  Expecting value: line 1 column 1 (char 0)
6    차례대로  Expecting value: line 1 column 1 (char 0)
7    감동받다  Expecting value: line 1 column 1 (char 0)
8     낚시줄  Expecting value: line 1 column 1 (char 0)
9     동시집  Expecting value: line 1 column 1 (char 0)
10   찌부리다  Expecting value: line 1 column 1 (char 0)
11     돼다  Expecting value: line 1 column 1 (char 0)
12     벨라  Expecting value: line 1 column 1 (char 0)
13    꿰메다  Expecting value: line 1 column 1 (char 0)
14  비뚫어지다  Expecting value: line 1 column 1 (char 0)
15   심술궃다  Expecting value: line 1 column 1 (char 0)
16   적당히다  Expecting value: line 1 column 1 (char 0)
17   전해오다  Expecting value: line 1 column 1 (c

In [8]:
print(filtered_df)

     Word  Level
0      가다      1
1      그림      1
2      되다      1
3     만들다      1
4      먹다      1
...   ...    ...
1947   판단      7
1949   하자      7
1951   호머      7
1952    홍      7
1953   효과      7

[1912 rows x 2 columns]


In [9]:
# 검색 안된 단어들 재필터링
new_rows = pd.DataFrame([
    {'Word': '바른', 'Level': 2},
    {'Word': '낚시', 'Level': 3},
    {'Word': '동시', 'Level': 3},
    {'Word': '찌푸리다', 'Level': 3},
    {'Word': '꿰다', 'Level': 4},
    {'Word': '삐뚤어지다', 'Level': 4},
    {'Word': '심술', 'Level': 4},
    {'Word': '적당하다', 'Level': 4},
    {'Word': '전하다', 'Level': 4},
    {'Word': '참여', 'Level': 5},
    {'Word': '참여하다', 'Level': 5},
    {'Word': '총명', 'Level': 5},
    {'Word': '총명하다', 'Level': 5}
])

# Append the new rows to the existing DataFrame
filtered_df = pd.concat([filtered_df, new_rows], ignore_index=True)


In [10]:
print(filtered_df)

      Word  Level
0       가다      1
1       그림      1
2       되다      1
3      만들다      1
4       먹다      1
...    ...    ...
1920   전하다      4
1921    참여      5
1922  참여하다      5
1923    총명      5
1924  총명하다      5

[1925 rows x 2 columns]


In [13]:
filtered_df = filtered_df.sort_values(by=['Level', 'Word']).reset_index(drop=True)

filtered_df = filtered_df.drop_duplicates(subset=['Word']).reset_index(drop=True)

In [14]:
print(filtered_df)

       Word  Level
0        가다      1
1        가다      1
2        가방      1
3        가위      1
4     가위바위보      1
...     ...    ...
1920     판단      7
1921     하자      7
1922     호머      7
1923      홍      7
1924     효과      7

[1925 rows x 2 columns]


In [15]:
filtered_df.to_csv("../data/processed/word_dictionary.csv", sep=',', encoding='utf-8-sig', index=False)

In [3]:
filtered_df = pd.read_csv("../data/processed/word_dictionary.csv", sep=',')
filtered_df = filtered_df.drop_duplicates(subset=['Word']).reset_index(drop=True)

filtered_df.to_csv("../data/processed/word_dictionary.csv", index=False, encoding='utf-8-sig')